# Generate SPOCK training data

In [1]:
import spock
import random
import numpy as np
import rebound
import pandas as pd
from spock import simsetup
from spock import FeatureClassifier

load dataset

In [2]:
#specify the data path
#We will be using cleaned data generated from the original spock initial conditions data, filtered according to https://github.com/Ethadhani/SPOCKcleanData.git
datapath = '../../cleanData/csvs/resonant/'
initial = pd.read_csv(datapath+'clean_initial_conditions.csv')
labels = pd.read_csv(datapath+'clean_labels.csv')
#drop junk column
initial = initial.drop('Unnamed: 0', axis = 1)
#merge labels and initial conditions based on runstring
Initialdataset = initial.set_index('runstring').join(labels.set_index('runstring'))
Initialdataset = Initialdataset.drop('Unnamed: 0', axis = 1)

We can establish a function that, given a list of initial conditions, will return a rebound simulation

In [3]:
def get_sim(row, dataset):
    '''Given a row number, and a data sheet containing initial conditions, returns a corresponding simulation
    
        Arguments:
            row: what row the simulation you would like to create is on
                format of row is in order: 
                [index, 'p0m', 'p0x', 'p0y', 'p0z', 'p0vx', 'p0vy', 'p0vz', 'p1m', 'p1x', 'p1y',
                'p1z', 'p1vx', 'p1vy', 'p1vz', 'p2m', 'p2x', 'p2y', 'p2z', 'p2vx',
                'p2vy', 'p2vz', 'p3m', 'p3x', 'p3y', 'p3z', 'p3vx', 'p3vy', 'p3vz']

            dataset: what dataset contains your initial conditions

        return: returns a rebound simulation with the specified initial conditions'''
    try:
        data = dataset.loc[row]
        sim = rebound.Simulation()
        sim.G=4*np.pi**2
        sim.add(m=data['p0m'], x=data['p0x'], y=data['p0y'], z=data['p0z'], vx=data['p0vx'], vy=data['p0vy'], vz=data['p0vz'])
        sim.add(m=data['p1m'], x=data['p1x'], y=data['p1y'], z=data['p1z'], vx=data['p1vx'], vy=data['p1vy'], vz=data['p1vz'])
        sim.add(m=data['p2m'], x=data['p2x'], y=data['p2y'], z=data['p2z'], vx=data['p2vx'], vy=data['p2vy'], vz=data['p2vz'])
        sim.add(m=data['p3m'], x=data['p3x'], y=data['p3y'], z=data['p3z'], vx=data['p3vx'], vy=data['p3vy'], vz=data['p3vz'])
        return sim
    except:
        print("Error reading initial condition {0}".format(row))
        return None

We can now generate the set of system row indices

In [4]:
#generates the indexes of the systems
systemNum = range(Initialdataset.shape[0])

We can note the column names and import the different feature generators

In [5]:
col = ['EMcrossnear', 'EMfracstdnear', 'EPstdnear', 'MMRstrengthnear', 'EMcrossfar', 'EMfracstdfar', 'EPstdfar', 'MMRstrengthfar', 'MEGNO', 'MEGNOstd','Tmax', 'InitialStable']

In [6]:
spock = FeatureClassifier()

We can then establish some helper functions that will allow us to map the spock.generate_feature function to the different systems by mapping to different row numbers and generating the correct simulation

In [7]:
def getList(features):
    '''Helper function which isolates the data list from the generate_features return'''
    return list(features[0][0].values())+[features[1]]

In [8]:
def getFeat(num):
    '''when given a index of a row, loads initial conditions and returns the spock generated features'''
    #gets features based on index num
    sim = get_sim(num,initial)
    return spock.generate_features(sim, Tmax = True)

In [9]:
rebound.__version__

'4.3.2'

We can now map getFeat to the different rows of the Initial df, this will create each simulation and generate the spock features.

In [10]:
%%time
import sys
from multiprocessing import Pool
sys.path.append('spockUpdate/train_models')
#import is required to get around jupyter notebook bug with multiprocessing
if __name__ == "__main__":
    with Pool() as pool:
        features = pool.map(getFeat,systemNum)
        pool.close()
        pool.join()
#formats the data correctly



CPU times: user 3.09 s, sys: 590 ms, total: 3.68 s
Wall time: 23min 20s


In [11]:
formattedFeat = pd.DataFrame(np.array(list(map(getList,features))), columns = col)

We can then join the generated features with the corresponding labels

In [12]:
dataset = pd.DataFrame.join(formattedFeat,labels)

In [13]:
dataset

,EMcrossnear,EMfracstdnear,EPstdnear,MMRstrengthnear,EMcrossfar,EMfracstdfar,EPstdfar,MMRstrengthfar,MEGNO,MEGNOstd,Tmax,InitialStable,Unnamed: 0,runstring,instability_time,shadow_instability_time,Stable
0,0.060234,0.029994,0.001160,0.513338,0.504063,0.001808,0.000559,NaN,1.991687,0.003411,20982.207221,1.0,0,0000000.bin,1.545872e+06,3.063700e+06,False
1,0.080547,0.017239,0.000138,0.432211,0.240504,0.008924,0.002141,0.008266,1.994730,0.003807,10000.000000,1.0,1,0000001.bin,9.990000e+08,9.990000e+08,True
2,0.129660,0.028790,0.001050,1.042844,1.001981,0.000465,0.000964,0.010449,1.975746,0.015904,10273.032450,1.0,2,0000002.bin,9.990000e+08,9.990000e+08,True
3,0.406112,0.038497,0.001201,0.267824,0.427768,0.037941,0.006196,0.013943,2.002463,0.000548,12772.209007,1.0,3,0000003.bin,2.287671e+06,8.392234e+06,False
4,0.059897,0.026167,0.001919,0.285146,0.257596,0.052339,0.001386,0.032904,1.805866,0.080916,10000.000000,1.0,4,0000004.bin,9.668931e+05,3.380350e+05,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102492,0.089252,0.275075,0.011026,0.046877,0.310694,0.033706,0.000990,1.102392,76.243870,20.441126,10123.428805,1.0,113537,9044761.bin,6.303165e+04,6.470086e+04,False
102493,0.082222,0.049940,0.016289,0.614960,0.664222,0.031330,0.003745,NaN,1.998822,0.003453,10000.000000,1.0,113538,9045377.bin,6.990387e+05,8.267916e+05,False
102494,0.131799,0.027696,0.001457,0.660450,0.366664,0.067480,0.020412,0.005440,7.135916,1.918209,10000.000000,1.0,113540,9045380.bin,1.193822e+07,3.363291e+07,False
102495,0.209454,0.090838,0.010214,1.438882,0.395073,0.102280,0.038515,0.041145,2.117164,0.088393,10000.000000,1.0,113541,9045382.bin,2.064407e+08,4.316851e+07,False


We can then save the new training data spreadsheet.

In [14]:
dataset.to_csv(datapath+'TmaxOr1e4.csv')